In [124]:
import numpy as np
import math
np.set_printoptions(precision=128)

functionCalculated = 0

def f(x):
    x1 = x[0]
    x2 = x[1]
    global functionCalculated
    functionCalculated += 1
    return x1**2 + x2**2
    # return (x1-1)**2 + (x2-1)**2
    # return 100 * (x1**2-x2)**2 + (x1-1)**2

In [125]:
# print(f"{h}, {h*np.array([1, 0])}, {h*np.array([0, 1])}")
def fderright(x, fx, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - fx) / h
    gradFx2 = (f(x + h*np.array([0, 1])) - fx) / h
    return np.array([gradFx1, gradFx2])

def fderleft(x, fx, h):
    gradFx1 = (fx - f(x - h*np.array([1, 0]))) / h
    gradFx2 = (fx - f(x - h*np.array([0, 1]))) / h
    return np.array([gradFx1, gradFx2])

def fdercenter(x, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - f(x - h*np.array([1, 0]))) / (2*h)
    gradFx2 = (f(x + h*np.array([0, 1])) - f(x - h*np.array([0, 1]))) / (2*h)
    return np.array([gradFx1, gradFx2])

In [126]:
def norm(x):
    x1 = x[0]
    x2 = x[1]
    return math.sqrt(x1**2 + x2**2)

def stopCriteria1(xk, fxk, xk_1, fxk_1, e):
    if ((norm((xk_1 - xk)) / norm(xk)) <= e) and (math.fabs(fxk_1 - fxk) <= e):
        return True
    return False

def stopCriteria2(gradFxk, e):
    if (norm(gradFxk) <= e):
        return True
    return False

In [127]:
# Code for finding optimal Lambda
functionCalculatedForLambda = 0

svenCoefLambda = 1
l0 = 0

eGoldenCut = 0.01

'''
def f(x):
    x1 = x[0]
    x2 = x[1]
    return 3*(x1-15)**2 -x1*x2 +4*x2**2
'''

def FLambda(xk, l, Sk):
    global functionCalculatedForLambda
    functionCalculatedForLambda += 1
    return f(xk + l*Sk)

def sven(xk, Sk):
    deltaL = svenCoefLambda * (norm(xk) / norm(Sk))
    FLambda0 = FLambda(xk, l0, Sk)
    FLambdaPlus = FLambda(xk, l0+deltaL, Sk)
    FLambdaMinus = FLambda(xk, l0-deltaL, Sk)
    
    coef = 0
    currentFValue = 0
    if FLambda0 < FLambdaPlus and FLambda0 < FLambdaMinus:
        print("Sven: both Lambdas are increasing funcion")
        return np.array([-deltaL, deltaL])
    if FLambda0 > FLambdaPlus and FLambda0 > FLambdaMinus:
        raise Exception("Sven: both Lambdas are deacreasing function")
    if FLambda0 > FLambdaPlus:
        coef = 1
        currentFValue = FLambdaPlus
        l1 = l0 + coef * deltaL
    if FLambda0 > FLambdaMinus:
        coef = -1
        currentFValue = FLambdaMinus
    l1 = l0 + coef * deltaL
    currentlValue = l1
    lambdaValues = [l0, l1]
    FNewLambda = 0
    n = 1
    while True:
        newLambda = currentlValue + 2*n*coef * deltaL
        lambdaValues.append(newLambda)
        FNewLambda = FLambda(xk, newLambda, Sk)
        if (currentFValue < FNewLambda):
            break
        currentlValue = newLambda
        currentFValue = FNewLambda
        n = 2*n
    print(f"SVEN - deltaL: {deltaL}, Lambdas: {lambdaValues}, functionCalculatedForLambda: {functionCalculatedForLambda}")
    return np.array([lambdaValues[-3], (lambdaValues[-2] + lambdaValues[-1]) / 2])

def goldenCut(lTrustInterval, xk, Sk):
    a = lTrustInterval[0]
    b = lTrustInterval[1]
    biggerCut = (math.sqrt(5) - 1) / 2
    smallerCut = 1 - biggerCut
    L = b - a
    x1 = a + smallerCut * L
    x2 = a + biggerCut * L
    Fx1 = FLambda(xk, x1, Sk)
    Fx2 = FLambda(xk, x2, Sk)

    while (L > eGoldenCut):
        if Fx1 < Fx2:
            b = x2
            x2 = x1
            L = b - a
            x1 = a + smallerCut * L
            Fx2 = Fx1
            Fx1 = FLambda(xk, x1, Sk)
        else:
            a = x1
            x1 = x2
            L = b - a
            x2 = a + biggerCut * L
            Fx1 = Fx2
            Fx2 = FLambda(xk, x2, Sk)
    print(f"GOLDENCUT - INTERVAL: {np.array([x1, x2])}, functionCalculatedForLambda: {functionCalculatedForLambda}")
    return np.array([x1, x2])

def DSKPauela():
    pass

# trustInterval = sven(np.array([-22, -22]), np.array([0, 1]))
# print(goldenCut(trustInterval, np.array([-22, -22]), np.array([0, 1])))
# print(functionCalculatedForLambda)
    

In [128]:
x0 = np.array([1, 1])

h = 0.0001
def MOPlconst(e):
    xk = x0
    l = 1
    Fxk = f(xk)
    countGrad = True
    while True:
        if countGrad:
            gradFxk = fderright(xk, Fxk, h)# fdercenter(xk, h)
        xk_new = xk + l * (-gradFxk / norm(gradFxk))
        Fxk_new = f(xk_new)
        
        print(f'{xk_new}, {l}, {Fxk}, {Fxk_new}')
        
        if stopCriteria1(xk, Fxk, xk_new, Fxk_new, e):
            return xk_new
        if (Fxk <= Fxk_new):
            # What should I do with L?
            l = l / 2
            countGrad = False
        else: 
            xk = xk_new
            Fxk = Fxk_new
            countGrad = True

# stopCriteria2(gradFxk, e)
def MOPlopt(e):
    i = 0
    xk = x0
    Fxk = f(xk)
    gradFxk = fdercenter(xk, h)
    while True:
        #  fderright(xk, Fxk, h) fdercenter(xk, h)
        i += 1
        print(f"\nITERATION NUMBER {i}")
        Sk = - gradFxk
        l = np.mean(goldenCut(sven(xk, Sk), xk, Sk))
        xk_new = xk + l * Sk
        Fxk_new = f(xk_new)
        
        oldGrad = gradFxk
        gradFxk = fdercenter(xk_new, h)
        print(f'Xk+1: {xk_new}, Lambda: {l}, Fxk: {Fxk}, Fxk+1: {Fxk_new}, GradFxk: {gradFxk}, GradFxk+1: {gradFxk}')
        
        if stopCriteria2(gradFxk, e):
            return xk_new
        xk = xk_new
        Fxk = Fxk_new
        

print(MOPlopt(0.01))
print(functionCalculated)
# print(functionCalculatedForLambda)


ITERATION NUMBER 1
SVEN - deltaL: 0.5000000000000551, Lambdas: [0, 0.5000000000000551, 1.5000000000001652], functionCalculatedForLambda: 4
GOLDENCUT - INTERVAL: [0.5015528100076261 0.5034721887331259], functionCalculatedForLambda: 16
Xk+1: [-0.00502499874064144 -0.00502499874064144], Lambda: 0.502512499370376, Fxk: 2, Fxk+1: 5.050122468689611e-05, GradFxk: [-0.010049997481282933 -0.010049997481282933], GradFxk+1: [-0.010049997481282933 -0.010049997481282933]

ITERATION NUMBER 2
SVEN - deltaL: 0.49999999999999734, Lambdas: [0, 0.49999999999999734, 1.499999999999992], functionCalculatedForLambda: 20
GOLDENCUT - INTERVAL: [0.5015528100075681 0.5034721887330679], functionCalculatedForLambda: 32
Xk+1: [2.5250612343448615e-05 2.5250612343448615e-05], Lambda: 0.502512499370318, Fxk: 5.050122468689611e-05, Fxk+1: 1.275186847438239e-09, GradFxk: [5.050122468689721e-05 5.050122468689721e-05], GradFxk+1: [5.050122468689721e-05 5.050122468689721e-05]
[2.5250612343448615e-05 2.5250612343448615e-05